In [15]:
import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [16]:
def fetch_range_chunked(start_date, end_date):
    max_days = 30  # chunk size
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    all_data = []
    
    while start <= end:
        chunk_end = min(start + timedelta(days=max_days - 1), end)
        s = start.strftime("%Y-%m-%dT00:00Z")
        e = chunk_end.strftime("%Y-%m-%dT23:30Z")
        
        print(f"Fetching data from {s} to {e}...")
        url = f"https://api.carbonintensity.org.uk/intensity/{s}/{e}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        records = []
        for item in data['data']:
            records.append({
                'from': pd.to_datetime(item['from']),
                'to': pd.to_datetime(item['to']),
                'forecast': item['intensity']['forecast'],
                'actual': item['intensity']['actual'],
                'index': item['intensity']['index']
            })
        
        df_chunk = pd.DataFrame(records)
        all_data.append(df_chunk)
        
        start = chunk_end + timedelta(days=1)
    
    df_all = pd.concat(all_data).reset_index(drop=True)
    df_all['day_of_week'] = df_all['to'].dt.day_name()
    return df_all

In [17]:
# PIVOT FUNCTION (optional - keep if you need 48HH format)
def pivot_to_48HH(df):
    df['date'] = df['to'].dt.date
    df = df.sort_values(['date', 'to']).reset_index(drop=True)
    df['half_hour_slot'] = df.groupby('date').cumcount() + 1
    df_pivot = df.pivot(index='date', columns='half_hour_slot', values='actual')
    df_pivot.columns = [f'hh_{i}' for i in range(1, 49)]
    df_pivot.reset_index(inplace=True)
    return df_pivot

In [18]:
# FETCH CONTINUOUS DATA FOR ENTIRE STUDY PERIOD
print("Fetching continuous carbon data for entire study period...")
carbon_continuous = fetch_range_chunked("2023-01-01", "2024-12-31")

print(f"Total carbon records fetched: {len(carbon_continuous)}")
print(f"Date range: {carbon_continuous['from'].min()} to {carbon_continuous['to'].max()}")
print(f"Missing actual values: {carbon_continuous['actual'].isna().sum()}")

# Save the continuous dataset
carbon_continuous.to_csv("carbon_continuous.csv", index=False)
print("Saved continuous carbon data to 'carbon_continuous.csv'")

Fetching continuous carbon data for entire study period...
Fetching data from 2023-01-01T00:00Z to 2023-01-30T23:30Z...
Fetching data from 2023-01-31T00:00Z to 2023-03-01T23:30Z...
Fetching data from 2023-03-02T00:00Z to 2023-03-31T23:30Z...
Fetching data from 2023-04-01T00:00Z to 2023-04-30T23:30Z...
Fetching data from 2023-05-01T00:00Z to 2023-05-30T23:30Z...
Fetching data from 2023-05-31T00:00Z to 2023-06-29T23:30Z...
Fetching data from 2023-06-30T00:00Z to 2023-07-29T23:30Z...
Fetching data from 2023-07-30T00:00Z to 2023-08-28T23:30Z...
Fetching data from 2023-08-29T00:00Z to 2023-09-27T23:30Z...
Fetching data from 2023-09-28T00:00Z to 2023-10-27T23:30Z...
Fetching data from 2023-10-28T00:00Z to 2023-11-26T23:30Z...
Fetching data from 2023-11-27T00:00Z to 2023-12-26T23:30Z...
Fetching data from 2023-12-27T00:00Z to 2024-01-25T23:30Z...
Fetching data from 2024-01-26T00:00Z to 2024-02-24T23:30Z...
Fetching data from 2024-02-25T00:00Z to 2024-03-25T23:30Z...
Fetching data from 2024-03

In [19]:
# Optional: Create separate weekday/Sunday files if still needed for other analyses
carbon_weekdays = carbon_continuous[carbon_continuous['day_of_week'] != 'Sunday'].copy()
carbon_sundays = carbon_continuous[carbon_continuous['day_of_week'] == 'Sunday'].copy()

carbon_weekdays.to_csv("carbon_weekdays.csv", index=False)
carbon_sundays.to_csv("carbon_sundays.csv", index=False)

print(f"Weekday records: {len(carbon_weekdays)}")
print(f"Sunday records: {len(carbon_sundays)}")

# Display sample data
print("\nSample of continuous carbon data:")
print(carbon_continuous.head())

Weekday records: 29966
Sunday records: 5000

Sample of continuous carbon data:
                       from                        to  forecast  actual index  \
0 2022-12-31 23:30:00+00:00 2023-01-01 00:00:00+00:00        75    65.0   low   
1 2023-01-01 00:00:00+00:00 2023-01-01 00:30:00+00:00        73    72.0   low   
2 2023-01-01 00:30:00+00:00 2023-01-01 01:00:00+00:00        63    80.0   low   
3 2023-01-01 01:00:00+00:00 2023-01-01 01:30:00+00:00        71    72.0   low   
4 2023-01-01 01:30:00+00:00 2023-01-01 02:00:00+00:00        76    65.0   low   

  day_of_week  
0      Sunday  
1      Sunday  
2      Sunday  
3      Sunday  
4      Sunday  


# Merging the data

In [2]:
# Step 0.1: Define date ranges
pre_start = pd.Timestamp("2023-02-01 00:00:00")
pre_end   = pd.Timestamp("2024-01-31 23:59:59")

post_start = pd.Timestamp("2024-04-01 00:00:00")
post_end   = pd.Timestamp("2024-12-31 23:59:59")

In [26]:
def process_carbon_data_for_peak_hours(carbon_df):
    """Process carbon data to keep 30-minute intervals for 4-7 PM hours, filtered for electricity data periods only"""
    # Make a copy to avoid SettingWithCopyWarning
    carbon_df = carbon_df.copy()
    
    # Convert 'from' column to datetime and remove timezone info for consistency
    carbon_df['from'] = pd.to_datetime(carbon_df['from']).dt.tz_localize(None)
    
    # Filter for periods where electricity data exists
    # Pre period: 2023-02-02 to 2024-01-31
    # Post period: 2024-04-01 to 2024-12-31
    pre_mask = (carbon_df['from'] >= pd.Timestamp("2023-02-02 00:00:00")) & \
               (carbon_df['from'] <= pd.Timestamp("2024-01-31 23:59:59"))
    
    post_mask = (carbon_df['from'] >= pd.Timestamp("2024-04-01 00:00:00")) & \
                (carbon_df['from'] <= pd.Timestamp("2024-12-31 23:59:59"))
    
    # Keep only data from periods where electricity data exists
    carbon_df = carbon_df[pre_mask | post_mask].copy()
    
    # Filter for 4-7 PM hours (16:00-18:30 to match your electricity filtering)
    carbon_df = carbon_df[(carbon_df['from'].dt.hour >= 16) & (carbon_df['from'].dt.hour < 19)].copy()
    
    # Keep the 30-minute intervals - create a datetime_30min column for exact matching
    carbon_df['datetime_30min'] = carbon_df['from']
    
    # Select only the columns we need
    carbon_30min = carbon_df[['datetime_30min', 'actual']].copy()
    carbon_30min.rename(columns={'actual': 'actual_intensity'}, inplace=True)
    
    return carbon_30min

In [27]:
def preprocess_electricity_with_carbon(file_name, start_date, end_date, treatment, period, carbon_data):
    """
    Process electricity data and merge with carbon data (keeping 30-minute intervals)
    """
    # Load and preprocess electricity data
    df = pd.read_csv(file_name)
    if 'Unnamed: 0' in df.columns:
        df = df.rename(columns={'Unnamed: 0': 'Time'})
    else:
        # Handle case where first column might have different name
        df.rename(columns={df.columns[0]: 'Time'}, inplace=True)
    
    # Convert Time column to datetime
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S")
    print(f"Processing {file_name} -> Min Time: {df['Time'].min()}, Max Time: {df['Time'].max()}")
    
    # Filter by time range (pre or post)
    df = df[(df['Time'] >= start_date) & (df['Time'] <= end_date)]
    
    # Keep only 4-7 PM consumption (matching your approach)
    df = df[(df['Time'].dt.hour >= 16) & (df['Time'].dt.hour < 19)]
    
    # Convert from wide to long format
    df_long = df.melt(id_vars=['Time'], var_name='ANON_ID', value_name='ELEC_KWH')
    
    # Remove duplicates after melting
    df_long.drop_duplicates(subset=['Time', 'ANON_ID'], inplace=True)
    
    # Create datetime_30min column for exact matching with carbon data
    df_long['datetime_30min'] = df_long['Time']
    
    # Merge with carbon data on exact 30-minute intervals
    df_with_carbon = pd.merge(df_long, carbon_data, on='datetime_30min', how='left')
    
    # Calculate carbon emissions
    df_with_carbon['carbon_emissions'] = df_with_carbon['ELEC_KWH'] * df_with_carbon['actual_intensity']
    
    # Add treatment and period columns
    df_with_carbon['treatment'] = treatment
    df_with_carbon['period'] = period
    
    # Create date column for easier analysis (but keep the 30-minute granularity)
    df_with_carbon['date'] = df_with_carbon['Time'].dt.date.astype(str)
    
    # Select and reorder final columns
    final_columns = ['ANON_ID', 'date', 'Time', 'ELEC_KWH', 'actual_intensity', 'carbon_emissions', 'treatment', 'period']
    df_final = df_with_carbon[final_columns].copy()
    
    return df_final

In [28]:
# Process Carbon Data

# Load the single continuous carbon file
carbon_continuous = pd.read_csv('carbon_continuous.csv')

In [29]:
# Process carbon data for peak hours (4-7 PM)
all_carbon = process_carbon_data_for_peak_hours(carbon_continuous)

print(f"Carbon data processed: {len(all_carbon)} 30-minute interval records")
print(f"Carbon datetime range: {all_carbon['datetime_30min'].min()} to {all_carbon['datetime_30min'].max()}")
print(f"Missing carbon intensity values: {all_carbon['actual_intensity'].isna().sum()}")

Carbon data processed: 3822 30-minute interval records
Carbon datetime range: 2023-02-02 16:00:00 to 2024-12-31 18:30:00
Missing carbon intensity values: 0


In [30]:
# Process Electricity Data with Carbon Integration

# Define your electricity files folder path
electricity_folder = "D:\LSE\Capstone Project\FLASH data" 

# Define file configurations: (filename, start_date, end_date, treatment, period)
# Excluding Sunday files
file_configs = [
    ("controlePreConsolide.csv", pre_start, pre_end, "Control", "Pre"),
    ("controleConsolide.csv", post_start, post_end, "Control", "Post"),
    ("interventionPreConsolide.csv", pre_start, pre_end, "Intervention", "Pre"),
    ("interventionConsolide.csv", post_start, post_end, "Intervention", "Post")
]

# Process all files in a loop (weekdays only)
processed_datasets = {}
dataset_names = ["df_control_pre", "df_control_post", 
                "df_intervention_pre", "df_intervention_post"]

for i, (filename, start_date, end_date, treatment, period) in enumerate(file_configs):
    file_path = os.path.join(electricity_folder, filename)
    dataset_name = dataset_names[i]
    
    processed_datasets[dataset_name] = preprocess_electricity_with_carbon(
        file_path, start_date, end_date, treatment, period, all_carbon)

# Extract individual datasets for easier access (weekdays only)
df_control_pre = processed_datasets["df_control_pre"]
df_control_post = processed_datasets["df_control_post"]
df_intervention_pre = processed_datasets["df_intervention_pre"]
df_intervention_post = processed_datasets["df_intervention_post"]

print("Individual dataset shapes:")
for name, df in processed_datasets.items():
    print(f"{name}: {df.shape}")

Processing D:\LSE\Capstone Project\FLASH data\controlePreConsolide.csv -> Min Time: 2023-02-02 00:00:00, Max Time: 2024-01-31 23:30:00
Processing D:\LSE\Capstone Project\FLASH data\controleConsolide.csv -> Min Time: 2024-04-01 00:00:00, Max Time: 2024-12-31 23:30:00
Processing D:\LSE\Capstone Project\FLASH data\interventionPreConsolide.csv -> Min Time: 2023-02-02 00:00:00, Max Time: 2024-01-31 23:30:00
Processing D:\LSE\Capstone Project\FLASH data\interventionConsolide.csv -> Min Time: 2024-04-01 00:00:00, Max Time: 2024-12-31 23:30:00
Individual dataset shapes:
df_control_pre: (691200, 8)
df_control_post: (713460, 8)
df_intervention_pre: (839790, 8)
df_intervention_post: (936396, 8)


In [31]:
# Combine all datasets (weekdays only)
all_datasets = [
    df_control_pre, df_control_post,
    df_intervention_pre, df_intervention_post
]

# Filter out empty datasets
all_datasets = [df for df in all_datasets if not df.empty]

# Combine all data
final_df = pd.concat(all_datasets, ignore_index=True)

# Reorder columns to match your desired format
final_df = final_df[['ANON_ID', 'date', 'ELEC_KWH', 'actual_intensity', 'carbon_emissions', 'treatment', 'period']]

# Remove rows where carbon data is missing
final_df = final_df.dropna(subset=['actual_intensity'])

print(f"\nFinal combined dataset shape: {final_df.shape}")
print(f"Date range: {final_df['date'].min()} to {final_df['date'].max()}")
print(f"Unique households: {final_df['ANON_ID'].nunique()}")

# Display sample
print("\nSample of final data:")
print(final_df.head())


Final combined dataset shape: (3172362, 7)
Date range: 2023-02-02 to 2024-12-31
Unique households: 1173

Sample of final data:
                                             ANON_ID        date  ELEC_KWH  \
0  4dd3317694364b953434dc42eff7f9950095c4ab537c3b...  2023-02-02     0.243   
1  4dd3317694364b953434dc42eff7f9950095c4ab537c3b...  2023-02-02     0.292   
2  4dd3317694364b953434dc42eff7f9950095c4ab537c3b...  2023-02-02     0.260   
3  4dd3317694364b953434dc42eff7f9950095c4ab537c3b...  2023-02-02     0.541   
4  4dd3317694364b953434dc42eff7f9950095c4ab537c3b...  2023-02-02     0.400   

   actual_intensity  carbon_emissions treatment period  
0             128.0            31.104   Control    Pre  
1             132.0            38.544   Control    Pre  
2             125.0            32.500   Control    Pre  
3             121.0            65.461   Control    Pre  
4             125.0            50.000   Control    Pre  


In [36]:
# Save the final dataset
final_df.to_csv('carbon_electricity_data.csv', index=False)
print("\nData saved to 'carbon_electricity_data.csv")


Data saved to 'carbon_electricity_data.csv


In [37]:
# Check missing values in final dataset
print(f"Missing electricity values: {final_df['ELEC_KWH'].isna().sum()}")
print(f"Missing carbon emissions: {final_df['carbon_emissions'].isna().sum()}")
print(f"Percentage of missing data: {final_df['ELEC_KWH'].isna().mean()*100:.2f}%")

Missing electricity values: 146841
Missing carbon emissions: 146841
Percentage of missing data: 4.63%


In [38]:
# Remove records with missing electricity data
final_df_clean = final_df.dropna(subset=['ELEC_KWH'])

print(f"Original dataset: {len(final_df):,} records")
print(f"Clean dataset: {len(final_df_clean):,} records")
print(f"Records removed: {len(final_df) - len(final_df_clean):,}")

Original dataset: 3,172,362 records
Clean dataset: 3,025,521 records
Records removed: 146,841


In [40]:
# Save the cleaned dataset
final_df_clean.to_csv('carbon_electricity_data_clean.csv', index=False)
print("Clean dataset saved to 'carbon_electricity_data_clean.csv'")

Clean dataset saved to 'carbon_electricity_data_clean.csv'
